In [1]:
import os
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import pandas as pd
import pprint
import datetime as dt

In [2]:
url_corona = "https://www.worldometers.info/coronavirus/#countries"
html_corona = requests.get(url_corona)
html_corona = html_corona.text
#html_corona

In [3]:
soup_corona = bs(html_corona, 'html.parser')

In [55]:
corona_table = soup_corona.find(id = "main_table_countries_today")

corona_table = str(corona_table)
corona_table

'<table class="table table-bordered table-hover main_table_countries" id="main_table_countries_today" style="width:100%;margin-top: 0px !important;display:none;">\n<thead>\n<tr>\n<th width="1%">#</th>\n<th width="100">Country,<br>Other</br></th>\n<th width="20">Total<br>Cases</br></th>\n<th width="30">New<br>Cases</br></th>\n<th width="30">Total<br>Deaths</br></th>\n<th width="30">New<br>Deaths</br></th>\n<th width="30">Total<br>Recovered</br></th>\n<th width="30">New<br>Recovered</br></th>\n<th width="30">Active<br/>Cases</th>\n<th width="30">Serious,<br/>Critical</th>\n<th width="30">Tot\xa0Cases/<br/>1M pop</th>\n<th width="30">Deaths/<br/>1M pop</th>\n<th width="30">Total<br/>Tests</th>\n<th width="30">Tests/<br/>\n<nobr>1M pop</nobr>\n</th>\n<th width="30">Population</th>\n<th style="display:none" width="30">Continent</th>\n<th width="30">1 Case<br/>every X ppl</th><th width="30">1 Death<br/>every X ppl</th><th width="30">1 Test<br/>every X ppl</th>\n</tr>\n</thead>\n<tbody>\n<tr 

In [56]:
corona_df = pd.read_html(corona_table, match ="br")

#replacing blanks with 0
corona_df.fillna(0,inplace = True)

#removing the total row)
corona_df = corona_df[~corona_df['Country,Other'].isin(['Total:', 'Total', 'total'])]
corona_df.to_csv('new_data.csv')
#replace items using wild card and regex to have that applied
corona_df["NewCases"].replace("\,","",regex = True, inplace = True)
corona_df["NewCases"].replace("\+","", regex = True, inplace = True)
corona_df["NewCases"] = corona_df["NewCases"].astype("float")
corona_df["NewDeaths"].replace("\,","",regex = True, inplace = True)
corona_df["NewDeaths"].replace("\+","", regex = True, inplace = True)
corona_df["NewDeaths"] = corona_df["NewDeaths"].astype("float")


#add date column for date data scrped
corona_df["Date"]= dt.datetime.today()

ValueError: No tables found matching pattern 'br'

In [ ]:
corona_df

In [8]:
# adding lat long info 
countries_df = pd.read_csv("country_lan_lng.csv",engine='python')
merged = corona_df.merge(countries_df, left_on='Country,Other', right_on='name')
df_used = merged.drop(countries_df.columns[0:-2], axis=1)

df_used.rename(columns={"Country,Other":"Country","Serious,Critical": "CriticalCases"}, inplace = True)
#replace items using wild card and regex to have that applied
df_used["NewCases"].replace("\,","",regex = True, inplace = True)
df_used["NewCases"].replace("\+","", regex = True, inplace = True)
df_used["ClosedCases"] = df_used["TotalDeaths"] +  df_used["TotalRecovered"]

df_used["ClosedDeathRate"] = df_used["TotalDeaths"]/df_used["ClosedCases"]
df_used["ClosedDeathRate"].fillna(0,inplace = True)

df_used["CurrentDeathRate"] = df_used["TotalDeaths"]/df_used["TotalCases"]
df_used["CurrentDeathRate"].fillna(0,inplace = True)

total_cases = df_used["TotalCases"].sum()
total_closed_cases = df_used["ClosedCases"].sum()

df_used["Weight"] = [x/total_closed_cases for x in df_used["ClosedCases"]]

# #checking if total weight is 1 
# validate_weight = df_used["Weight"].sum()


df_used["DeathRateXWeight"] = df_used["CurrentDeathRate"] *df_used["Weight"]
df_used["DeathRateXWeight"].sum()
df_used["notes"] = ["Last 8 columns are added by Roaya" for x in df_used["DeathRateXWeight"]  ]
df_used.sort_values(by = "DeathRateXWeight", ascending = False)

NameError: name 'corona_df' is not defined

In [8]:
df_used.to_csv('data.csv')
df_used.to_html('data.html')

In [9]:
corona_json = df_used.to_json(orient='records')
df_used.to_json('data.json',orient='records')

In [10]:
corona_json

'[{"#":1.0,"Country":"USA","TotalCases":1686436,"NewCases":0.0,"TotalDeaths":99300.0,"NewDeaths":0.0,"TotalRecovered":451702.0,"ActiveCases":1135434.0,"CriticalCases":17135.0,"Tot\\u00a0Cases\\/1M pop":5098.0,"Deaths\\/1M pop":300.0,"TotalTests":14749756.0,"Tests\\/ 1M pop":44587.0,"Population":330806424.0,"Date":1590384432858,"Longitude":-97.922211,"Latitude":39.381266,"ClosedCases":551002.0,"ClosedDeathRate":0.1802171317,"CurrentDeathRate":0.0588815704,"Weight":0.2078565727,"DeathRateXWeight":0.0122389214,"notes":"Last 8 columns are added by Roaya"},{"#":2.0,"Country":"Brazil","TotalCases":365213,"NewCases":1595.0,"TotalDeaths":22746.0,"NewDeaths":30.0,"TotalRecovered":149911.0,"ActiveCases":192556.0,"CriticalCases":8318.0,"Tot\\u00a0Cases\\/1M pop":1719.0,"Deaths\\/1M pop":107.0,"TotalTests":735224.0,"Tests\\/ 1M pop":3461.0,"Population":212405664.0,"Date":1590384432858,"Longitude":-53.09783113,"Latitude":-10.78777702,"ClosedCases":172657.0,"ClosedDeathRate":0.1317409662,"CurrentDea